In [1]:
from datasets import Dataset, load_from_disk

/home/sai.prabhakar/LearnFromHumanEdit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
dataset = load_from_disk("data/avs/" + "sub_eval")

In [24]:
from typing import Dict, Optional
import random
import copy
from trainer import extract_prompt

def delete_random_words(sentence, max_words_to_delete=10) -> Dict[str, str]:
    # skip_deleting = random.randint(1, 10) > 7
    # if skip_deleting:
    #     return sentence

    words = sentence.split()    
    num_words_to_delete = random.randint(3, min(round(len(words)/3), max_words_to_delete))
    words_idx_to_delete = [random.randint(1, len(words) - 1) for _ in range(num_words_to_delete)]
    words = [word for idx, word in enumerate(words) if idx not in words_idx_to_delete]
    new_sentence = " ".join(words)
    return new_sentence

def insert_errors(sample) -> Dict[str, str]:
    prompt = extract_prompt(sample["chosen"])
    prompt = prompt
    chosen = sample["chosen"][len(prompt) :]
    rejected = sample["rejected"][len(prompt) :]

    rejected = delete_random_words(rejected)
    chosen = delete_random_words(chosen)
    new_sample = copy.deepcopy(sample)

    new_sample["rejected"] = prompt + " " + rejected
    new_sample["chosen"] = prompt + " " + chosen

    return new_sample

errored_dataset = dataset.map(insert_errors)

Map: 100%|██████████| 128/128 [00:00<00:00, 1720.25 examples/s]


In [25]:
errored_dataset.save_to_disk("data/avs/" + "sub_eval_w_simulated_edits")

Saving the dataset (1/1 shards): 100%|██████████| 128/128 [00:00<00:00, 24366.67 examples/s]


In [26]:
for i in range(10):
    print(errored_dataset[i]["rejected"] == errored_dataset[i]["chosen"])
    # print(dataset[i]["rejected"])
    # print(dataset[i]["chosen"])
    # print()

False
False
False
False
False
False
False
False
False
False


In [31]:
from sequence_alignment import needle


def get_position_status1(al_generated, 
                        al_edited):
    assert len(al_generated)==len(al_edited)
    
    position_status = []
    for i in range(len(al_generated)):
        if al_generated[i] == al_edited[i]:
            position_status.append('C')
            # S_generated_position_weight.append(S_generated_C_weight)
            # S_edited_position_weight.append(S_edited_C_weight)
        elif al_generated[i] == -100:
            position_status.append('I')
            # S_edited_position_weight.append(S_edited_I_weight)
        elif al_edited[i] == -100:
            position_status.append('D')
            # S_generated_position_weight.append(S_generated_D_weight)
        else:
            # print("S", al_generated[i], al_edited[i])
            position_status.append('S')
            # S_generated_position_weight.append(S_generated_S_weight)
            # S_edited_position_weight.append(S_edited_S_weight)
            
    return position_status

S_generated = [1, 2, 3, 4, 5, 6, 7, 8, 9]
S_edited = [1, 2, 3, 4, 5, 6, 10, 9]

alignment = needle.NeedlemanWunsch(S_generated, S_edited)
alignment.gap_character = -100
alignment.align()
alignment.get_score()
al_generated, al_edited = alignment.get_aligned_sequences("list_of_int")

p_status = get_position_status1(al_generated, al_edited)
print(alignment)
print(p_status)

Alignment of SEQUENCE 1 and SEQUENCE 2:
	123456789
	123456-100109

['C', 'C', 'C', 'C', 'C', 'C', 'D', 'S', 'C']


In [1]:
from dpo import *

/home/sai.prabhakar/LearnFromHumanEdit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sai.prabhakar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


SFTTrainer()

In [ ]:
SFTTrainer()

In [2]:
BaseTrainer.__doc__

"\n    Initialize DPOTrainer.\n\n    Args:\n        model (`transformers.PreTrainedModel`):\n            The model to train, preferably an `AutoModelForSequenceClassification`.\n        output_dir (`str`, defaults to `'./'`):\n        args (`transformers.TrainingArguments`):\n            The arguments to use for training.\n        data_collator (`transformers.DataCollator`):\n            The data collator to use for training. If None is specified, the default data collator (`DPODataCollatorWithPadding`) will be used\n            which will pad the sequences to the maximum length of the sequences in the batch, given a dataset of paired sequences.\n        label_pad_token_id (`int`, defaults to `-100`):\n            The label pad token id. This argument is required if you want to use the default data collator.\n        padding_value (`int`, defaults to `0`):\n            The padding value. This argument is required if you want to use the default data collator.\n        truncation_mode (`

In [ ]:
DPOTrainer()